In [ ]:
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print('GPU is available.')
else:
    print('GPU is NOT available. Make sure TensorFlow is installed with GPU support and NVIDIA GPU drivers are installed.')

# Define and configure TensorFlow session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import mimetypes
import argparse
import imutils
import os

In [ ]:
import cv2
image = cv2.imread("train\DJI_0019_R_JPG_jpg.rf.d5882396821ec4d18c77bb8224b526ce.jpg")
cv2.rectangle(image, (226, 73), (317, 129), (0, 255,0), 2)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)
plt.show()

In [ ]:
IMAGES_PATH = os.path.sep.join(["train"])

# Define the path to the CSV file containing annotations for bounding boxes
ANNOTS_PATH = os.path.sep.join(["_annotations.csv"])

In [ ]:
# Define the base output directory
BASE_OUTPUT = "output"

# Create the base output directory if it doesn't exist
os.makedirs(BASE_OUTPUT, exist_ok=True)

# Define the path to save the trained model
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.h5"])

# Define the path to save the plot of training history
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])

# Define the path to save the list of test image filenames
TEST_FILENAMES = os.path.sep.join([BASE_OUTPUT, "test_images.txt"])

In [ ]:
# Read the contents of the annotations CSV file and split it into rows
rows = open(ANNOTS_PATH).read().strip().split("\n")

# Initialize lists to store data, targets, and filenames
data = []
targets = []
filenames = []

In [ ]:
rows[:5]

In [ ]:
# Iterate through each row in the annotations data starting from the second row
for row in rows[1:]:
    # Split the row into individual values
    row = row.split(",")
    # Extract filename, bounding box coordinates (startX, startY, endX, endY)
    (filename,width,height,test,xmin,ymin,xmax,ymax) = row
    # Construct the path to the image
    imagePath = os.path.join(IMAGES_PATH, filename)
    # Read the image using OpenCV
    image = cv2.imread(imagePath)
    # Get the height and width of the image
    # Normalize the bounding box coordinates
    print(xmin,ymin,xmax,ymax)
    xmin = int(xmin) / int(width)
    ymin = int(ymin) / int(height)
    xmax = int(xmax) / int(width)
    ymax = int(ymax) / int(height)
    # Load and preprocess the image using Keras
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    # Append the preprocessed image, target coordinates, and filename to their respective lists
    data.append(image)
    targets.append((xmin, ymin, xmax, ymax))
    filenames.append(filename)

In [ ]:
# Convert the list of images to a NumPy array and normalize pixel values
data = np.array(data, dtype="float32") / 255.0

# Convert the list of targets to a NumPy array
targets = np.array(targets, dtype="float32")

# Split the data into training and testing sets along with filenames
split = train_test_split(data, targets, filenames, test_size=0.10, random_state=42)
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]

# Save the list of testing filenames to a file
with open(TEST_FILENAMES, "w") as f:
    f.write("\n".join(testFilenames))

In [ ]:
vgg = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Freeze the weights of the pretrained VGG16 model
vgg.trainable = False

# Get the output of the VGG16 model
flatten = vgg.output

# Flatten the output of the VGG16 model
flatten = Flatten()(flatten)

# Add fully connected layers for bounding box regression
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)

# Create a model that takes VGG16's input and predicts bounding box coordinates
model = Model(inputs=vgg.input, outputs=bboxHead)

In [ ]:
# Initialize the learning rate
INIT_LR = 1e-4

# Define the number of epochs for training
NUM_EPOCHS = 35

# Define the batch size for training
BATCH_SIZE = 16

In [ ]:
# Initialize the Adam optimizer with the specified learning rate
opt = Adam(learning_rate=INIT_LR)

# Compile the model with mean squared error loss and the Adam optimizer
model.compile(loss="mean_squared_error", optimizer=opt, metrics=["accuracy"])


# Print the summary of the model architecture
print(model.summary())

In [ ]:
# Train the model on the training data with validation on the testing data
H = model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    shuffle=True,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=1
)

In [ ]:
# Train the model on the training data with validation on the testing data
model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    shuffle=True,
    batch_size=BATCH_SIZE,
    epochs=20,
    verbose=1
)

In [ ]:
model.save('thermal_v1.h5')

In [ ]:
MODEL_PATH = "thermal_v1.h5"
model = load_model(MODEL_PATH)

In [ ]:
# Define the path to the testing image
imagePath = r"Y:\_Prjs\solar\testingfolders\ourtest\IMG-20240324-WA0019.jpg"

# Load and preprocess the testing image
image = load_img(imagePath, target_size=(224, 224))
image = img_to_array(image) / 255.0
image = np.expand_dims(image, axis=0)

# Make predictions using the trained model
preds = model.predict(image)[0]

# Extract predicted bounding box coordinates
(startX, startY, endX, endY) = preds

# Read the original image and resize it
image = cv2.imread(imagePath)
image = imutils.resize(image, width=600)
(h, w) = image.shape[:2]

# Scale the predicted coordinates to match the resized image
startX = int(startX * w)
startY = int(startY * h)
endX = int(endX * w)
endY = int(endY * h)

# Draw the bounding box on the image
cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

# Display the image with the bounding box
plt.imshow(image)